In [10]:
import pandas as pd
import re
import os
import chardet

# ===== 0) 경로 설정 =====
input_file  = "/Users/mac/Documents/SORA_Project/data/raw/Y06_서울시 안심귀갓길 안전시설물.csv"
output_dir  = "/Users/mac/Documents/SORA_Project/data/preprocessing"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "Y06_전처리_서울시 안심귀갓길 안전시설물.csv")

# ===== 1) 인코딩 자동 감지 및 파일 로드 =====
with open(input_file, "rb") as f:
    enc = chardet.detect(f.read(10000))["encoding"]

# 구분자 자동 감지
with open(input_file, "r", encoding="cp949", errors="ignore") as f:
    sample = f.readline()
sep = "\t" if "\t" in sample else ","

df = pd.read_csv(input_file, sep=sep, encoding="cp949")
print(f"\n✅ [1단계] 파일 로드 완료: {df.shape[0]:,}행 × {df.shape[1]}열")
print("📌 원본 컬럼:", df.columns.tolist())
df.info()


✅ [1단계] 파일 로드 완료: 11,883행 × 18열
📌 원본 컬럼: ['포인트 wkt', '시설물 id', '시군구 코드', '시군구명', '읍면동 코드', '읍면동명', '시설코드', '안심귀갓길 아이디', '안심귀갓길 명', '설치대수', '비고', '관리기관', '전화번호', '조성년월', '시설물 최종점검일(사용안함)', '세부위치설명', '데이터 기준일자', '이미지명']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11883 entries, 0 to 11882
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   포인트 wkt          11883 non-null  object 
 1   시설물 id           11883 non-null  object 
 2   시군구 코드           11883 non-null  int64  
 3   시군구명             11883 non-null  object 
 4   읍면동 코드           11883 non-null  int64  
 5   읍면동명             11883 non-null  object 
 6   시설코드             11883 non-null  int64  
 7   안심귀갓길 아이디        11883 non-null  object 
 8   안심귀갓길 명          11883 non-null  object 
 9   설치대수             11883 non-null  int64  
 10  비고               1138 non-null   object 
 11  관리기관             4289 non-null   object 
 12  전화번호             4172 n

In [15]:
# ===== 2) 시, 구, 동 추출 함수 =====
def extract_si(gu_name):
    if pd.isna(gu_name):
        return None
    return "서울특별시" if "서울" in str(gu_name) else None

def extract_gu(gu_name):
    if pd.isna(gu_name):
        return None
    match = re.search(r"(\S+구)", str(gu_name))
    return match.group(1) if match else None

# ===== 3) 시, 구, 동 컬럼 생성 =====
df["시"] = df["시군구명"].apply(extract_si)
df["구"] = df["시군구명"].apply(extract_gu)
df["동"] = df["읍면동명"]

# ===== 4) 시설코드 매핑 =====
facility_map = {
    301: "안심벨",
    302: "CCTV",
    303: "안내표지판(전주포함)",
    304: "노면표기",
    305: "보안등",
    306: "안심귀갓길 안내판",
    307: "112 위치신고 안내",
    308: "기타"
}
df["시설종류"] = df["시설코드"].map(facility_map)

# ===== 5) 동 기준 시설 수 집계 =====
df_grouped = (
    df.groupby(["시", "구", "동", "시설종류"])
    .size()
    .reset_index(name="시설 수")
)

# ===== 6) 구 단위 정렬 (오름차순) =====
df_grouped = df_grouped.sort_values(by=["구", "동", "시설종류"]).reset_index(drop=True)

# ===== 6-1) 시·구·동·시설종류·시설 수 결측치 확인 =====
cols_to_check = ["시", "구", "동", "시설종류"]
missing_counts = df_grouped[cols_to_check].isna().sum()
print("\n🔍 지정 컬럼 결측치:\n", missing_counts)

null_rows = df_grouped[df_grouped[cols_to_check].isna().any(axis=1)]
print(f"🚨 결측치 포함 행 수: {len(null_rows)}")
if len(null_rows) > 0:
    print(null_rows.head(10))

# ===== 7) 결과 확인 =====
print("\n✅ 결과 데이터 (상위 20행)")
print(df_grouped.head(20))

# ===== 6) 저장 =====
output_path = os.path.join(output_dir, "Y06_전처리_서울시 안심귀갓길 안전시설물.csv")
df_grouped.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"\n💾 집계 결과 저장 완료: {output_path}")


🔍 지정 컬럼 결측치:
 시       0
구       0
동       0
시설종류    0
dtype: int64
🚨 결측치 포함 행 수: 0

✅ 결과 데이터 (상위 20행)
        시    구    동         시설종류  시설 수
0   서울특별시  강남구  개포동  112 위치신고 안내     6
1   서울특별시  강남구  개포동         CCTV     3
2   서울특별시  강남구  개포동         노면표기     1
3   서울특별시  강남구  개포동          보안등    12
4   서울특별시  강남구  개포동          안심벨     2
5   서울특별시  강남구  논현동  112 위치신고 안내    27
6   서울특별시  강남구  논현동         CCTV    41
7   서울특별시  강남구  논현동         노면표기    18
8   서울특별시  강남구  논현동          보안등   122
9   서울특별시  강남구  논현동  안내표지판(전주포함)     8
10  서울특별시  강남구  논현동          안심벨    24
11  서울특별시  강남구  대치동  112 위치신고 안내    11
12  서울특별시  강남구  대치동         CCTV     7
13  서울특별시  강남구  대치동         노면표기     4
14  서울특별시  강남구  대치동          보안등    24
15  서울특별시  강남구  대치동          안심벨     7
16  서울특별시  강남구  도곡동  112 위치신고 안내     4
17  서울특별시  강남구  도곡동         CCTV     2
18  서울특별시  강남구  도곡동         노면표기     2
19  서울특별시  강남구  도곡동          보안등    13

💾 집계 결과 저장 완료: /Users/mac/Documents/SORA_Project/data/preprocessing/Y06_전처리_서